<a href="https://colab.research.google.com/github/abhilasha-kumar/fluency-cogsci2022/blob/main/fluency_cogsci2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phonological Intrusions in Semantic Memory Retrieval

# Importing drive, GPU, and packages

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Found GPU at: /device:GPU:0
Fri Jan  7 18:20:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    33W / 250W |    375MiB / 16280MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import heapq
import itertools
import scipy.spatial.distance

import pandas as pd
import numpy as np

from numpy.random import randint
from scipy.special import softmax
from sklearn.preprocessing import MinMaxScaler, normalize
from numpy.linalg import matrix_power
from functools import lru_cache
import glob
from scipy.special import expit

import matplotlib.pyplot as plt
import nltk
from functools import lru_cache
from itertools import product as iterprod
import itertools
from nltk.metrics import *



# Phoneme Function

In [4]:
# algo to obtain phonemes for any given strng
# obtained from: https://stackoverflow.com/questions/33666557/get-phonemes-from-any-word-in-python-nltk-or-other-modules
try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

@lru_cache()
def wordbreak(s):
    s = s.lower()
    if s in arpabet:
        return arpabet[s]
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabet[pre], wordbreak(suf))]
    return None

def normalized_sim(w1, w2):
  return 1-edit_distance(w1,w2)/(max(len(w1), len(w2)))


## example

In [5]:
w1 = "birds"
w2 = "pigs"
print("wordbreak(w1)[0]:",wordbreak(w1)[0])
print("wordbreak(w2)[0]:",wordbreak(w2)[0])

print("orig phon:", edit_distance(wordbreak(w1)[0],wordbreak(w2)[0]))
print("orig orth:", edit_distance(w1, w2))

print("norm orth:", normalized_sim(w1, w2))
print("norm phon:", normalized_sim(wordbreak(w1)[0],wordbreak(w2)[0]))

wordbreak(w1)[0]: ['B', 'ER1', 'D', 'Z']
wordbreak(w2)[0]: ['P', 'IH1', 'G', 'Z']
orig phon: 3
orig orth: 3
norm orth: 0.4
norm phon: 0.25


# reading data

In [17]:
parentfolder = "/content/drive/My Drive/IU-Abhilasha-Mike/Fluency/sem-phon/fluency_cogsci2022"
with tf.device('/device:GPU:0'):
  semantic_files = glob.glob(parentfolder + '/*.xlsx')
print(f"This folder has {len(semantic_files)} files")

This folder has 1 files


# reading embeddings

embeddings obtained from: http://vectors.nlpl.eu/explore/embeddings/en/models/

In [7]:
# import glove embeddings
parentfolder = "/content/drive/My Drive/IU-Abhilasha-Mike/Fluency/sem-phon/fluency_cogsci2022/"
with tf.device('/device:GPU:0'):
  word2vec_model = pd.read_csv(parentfolder +"wikimodel.csv", encoding="unicode-escape")
  word2vec = word2vec_model.transpose()
  new_header = word2vec.iloc[0] #grab the first row for the header
  word2vec = word2vec[1:] #take the data less the header row
  word2vec.columns = new_header
  word2vec = word2vec.values.transpose()
  vocab = pd.DataFrame(list(new_header), columns=["vocab_word"])
  print(f"embeddings are shaped:", word2vec.shape)
  print(f"vocab is {len(vocab)} words")

embeddings are shaped: (249211, 300)
vocab is 0 words


# obtaining phonemic & semantic similarity

In [42]:
## now we loop through each txt file
import warnings
warnings.filterwarnings("ignore")

phon_list = []

# read in the data as a pandas dataframe
data_file = pd.read_excel(semantic_files[0])
import re
for index, row in data_file.iterrows():  
  word = str(row["response"])
  mod_word = re.sub('[^a-zA-Z]+', '', word)
  if(len(mod_word)>0):
    phonemes = wordbreak(mod_word)[0]
    phon_list.append(phonemes)
  else:
    phon_list.append("wordnotfound")

data_file["phonemes"] = phon_list
data_file["response_number"] = data_file.groupby(['subject', 'domain']).cumcount()+1

# exclude rows that do not have a valid phoneme
data_file = data_file[data_file.phonemes != "wordnotfound"]
data_file = data_file.reset_index(drop= True)

#now we compute the levenshtein edit distance as a measure of orthographic/phonemic similarity 

phon_similarity = []
orth_similarity = []
glove_similarity = []

norm_phon = []
norm_orth = []

for index, row in data_file.iterrows():
  current_word = re.sub('[^a-zA-Z]+', '', str(row["response"]))
  current_phoneme = row["phonemes"]
  current_word = "FALSE" if current_word in ["False", "false"] else current_word
  if row["response_number"] == 1:
      sem_val = -999
      phon_val = -999
      orth_val = -999
      norm_phon_val = -999
      norm_orth_val = -999
  else:
    previous_word = re.sub('[^a-zA-Z]+', '', str(data_file.response[index-1]))
    previous_phoneme = data_file.phonemes[index-1]

    #calculate orthographic similarity as Levenshtein (edit) distance
    orth_val = edit_distance(previous_word, current_word)
    norm_orth_val = normalized_sim(previous_word, current_word)
    
    # can also get edit distance for the phonemes themselves (as in Siew et al. Hoosier network)

    phon_val = edit_distance(previous_phoneme, current_phoneme)
    norm_phon_val = normalized_sim(previous_phoneme, current_phoneme)
    
    # extract word embedding for current word
    if current_word in list(vocab["vocab_word"]):
      current_word_index = list(vocab["vocab_word"]).index(current_word)
      current_word_vec = word2vec[current_word_index].reshape((1,word2vec.shape[1]))
      # extract word embedding for current word
      if previous_word in list(vocab["vocab_word"]):
        previous_word_index = list(vocab["vocab_word"]).index(previous_word)
        previous_word_vec = word2vec[previous_word_index].reshape((1,word2vec.shape[1]))
        sem_val = float((1 - scipy.spatial.distance.cdist(previous_word_vec, current_word_vec, 'cosine'))[0])      
        #print(f"for {current_word} and {previous_word} similarity is {sem_val}")
      else:
        sem_val = "NA"
    else:
      sem_val = "NA"

  phon_similarity.append(phon_val)
  orth_similarity.append(orth_val)
  norm_phon.append(norm_phon_val)
  norm_orth.append(norm_orth_val)
  glove_similarity.append(sem_val)
  
data_file["phon_similarity"] = phon_similarity
data_file["orth_similarity"] = orth_similarity
data_file["norm_phon"] = norm_phon
data_file["norm_orth"] = norm_orth
data_file["word2vec_similarity"] = glove_similarity 
data_file.to_csv(parentfolder + f'precomputed_data.csv')

In [43]:
data_file

,dataset,subject,domain,response_number,response,response_onset_time,IRT,participant_designated_switch,phonemes,phon_similarity,orth_similarity,norm_phon,norm_orth,word2vec_similarity
0,LEA,50001,animals,1,horse,2.594,2.594000,2.0,"[HH, AO1, R, S]",-999,-999,-999.000,-999.000000,-999
1,LEA,50001,animals,2,pig,3.594,1.000000,0.0,"[P, IH1, G]",4,5,0.000,0.000000,0.132521
2,LEA,50001,animals,3,bear,4.894,1.300000,1.0,"[B, EH1, R]",3,4,0.000,0.000000,0.196242
3,LEA,50001,animals,4,cat,6.194,1.300000,1.0,"[K, AE1, T]",3,3,0.000,0.250000,0.1052
4,LEA,50001,animals,5,dog,7.394,1.200000,0.0,"[D, AO1, G]",3,3,0.000,0.000000,0.202481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32397,HJT,1099,sports,24,jumprope,130.000,46.308824,NaN,"[JH, AH1, M, P, R, OW1, P]",7,7,0.125,0.222222,NA
32398,HJT,1099,sports,25,hockey,163.000,33.926471,NaN,"[HH, AA1, K, IY0]",7,8,0.000,0.000000,NA
32399,HJT,1099,sports,26,fieldhockey,169.000,3.529412,NaN,"[F, IY1, L, D, HH, AA1, K, IY0]",4,5,0.500,0.545455,NA
32400,HJT,1099,sports,27,lacrosse,171.000,3.852941,NaN,"[L, AH0, K, R, AO1, S]",7,9,0.125,0.181818,NA


In [46]:
vec1 = word2vec[list(vocab["vocab_word"]).index("boy")].reshape((1,word2vec.shape[1]))
vec2 = word2vec[list(vocab["vocab_word"]).index("man")].reshape((1,word2vec.shape[1]))
float((1 - scipy.spatial.distance.cdist(vec1, vec2, 'cosine'))[0])      

0.19650142429788575

In [47]:
def cosine_similarity(word1, word2):
  vec1 = word2vec[list(vocab["vocab_word"]).index(word1)].reshape((1,word2vec.shape[1]))
  vec2 = word2vec[list(vocab["vocab_word"]).index(word2)].reshape((1,word2vec.shape[1]))
  return float((1 - scipy.spatial.distance.cdist(vec1, vec2, 'cosine'))[0])      

In [49]:
cosine_similarity("apple", "apple")

0.9999999999999999

In [60]:
def find_closest(word):
  sims = [cosine_similarity(word,x) for x in list(vocab.vocab_word)[:1000]]
  y = np.array(sims)
  y_sorted = np.argsort(-y) ## gives sorted indices
  top5_indices = y_sorted[:5]
  w1 = [list(vocab.vocab_word)[i] for i in top5_indices]
  return w1

In [61]:
find_closest("apple")

['great', 'US', 'main', 'car', 'flight']

# computational search models

we only consider the "animals" domain for the computational models. we start with a predefined list of 675 animals for which we create a semantic and phonological similarity matrix and obtain word frequency estimates.

# download animals list

In [64]:
simlabels = pd.read_csv(parentfolder+ f'similaritylabels.csv').values.reshape(-1,).tolist()
len(simlabels)

675

In [71]:
'blue jay' in list(vocab.vocab_word)

False

## create semantic similarity matrix

In [81]:
## create similarity matrix and similarity labels file from whichever corpus you're using

def create_sim_matrix(simlabels):
  ## the vocab consists of ALL possible words in corpus, but we need only the "animals" subset here
  ## we use the similarity_labels file from psyrev to constrain the vocab
  animals_index = [list(vocab.vocab_word).index(lab) if lab in list(vocab.vocab_word) else -999 for lab in simlabels ]
  animals_index = list(filter((-999).__ne__, animals_index))

  ## now we restrict our vocab and embeddings to ONLY these animals
  embeddings_small = word2vec[animals_index, :]
  vocab_small = vocab.iloc[animals_index]
  N = len(vocab_small)
  print(f"animals vocab is {N} words")

  # create semantic similarity matrix
  matrix = 1-scipy.spatial.distance.cdist(embeddings_small, embeddings_small, 'cosine').reshape(-1)
  matrix = matrix.reshape((N,N))
  print("sim matrix has been created:", matrix.shape)

  w1_index = list(vocab_small.vocab_word).index("dolphin")
  w2_index = list(vocab_small.vocab_word).index("kitten")
  w3_index = list(vocab_small.vocab_word).index("whale")

  print("dolphin-kitten:", matrix[w1_index, w2_index])
  print("dolphin-whale:", matrix[w1_index, w3_index])
  print("dolphin-dolphin:", matrix[w1_index, w1_index])

  pd.DataFrame(matrix).to_csv(parentfolder + 'word2vec_sim_matrix.csv', index=False, header=False)
  vocab_small.to_csv(parentfolder + 'word2vec_sim_labels.csv', index=False, header=False)

  print("sim matrix has been saved to drive!")

create_sim_matrix(simlabels)


animals vocab is 423 words
sim matrix has been created: (423, 423)
dolphin-kitten: 0.0742156239385755
dolphin-whale: 0.03625136598448808
dolphin-dolphin: 0.9999999999999999
sim matrix has been saved to drive!


## create phonological similarity matrix

In [86]:
def create_phon_matrix(vocab):
  # takes in a list of labels and computes the phonological similarity matrix
  vocabulary = vocab.copy()
  N = len(vocabulary)
  print(f"vocab is {N} words")
  # replace all underscores (_) with space (" ") to match with glove vectors/vocab
  vocabulary = [re.sub('[^a-zA-Z]+', '', str(v)) for v in vocabulary]
  print(f"vocab now looks like:", vocabulary[:5])
  # create phonemic similarity matrix for the small vocab
  pmatrix = np.array([normalized_sim(wordbreak(w1)[0], wordbreak(w2)[0]) for w1 in vocabulary for w2 in vocabulary]).reshape((N,N))
  print("pmatrix has been created:", pmatrix.shape)
  print(pmatrix)
  pd.DataFrame(pmatrix).to_csv(parentfolder + 'simlabels_phon_matrix.csv', index=False, header=False)  
  print("phon matrix has been saved to drive!")

simlabels = pd.read_csv(parentfolder+'word2vec_sim_labels.csv', header=None).values.reshape(-1,).tolist()
print(f"simlabels is {len(simlabels)} items:", simlabels[:5])
create_phon_matrix(simlabels)


simlabels is 423 items: ['Persian', 'Spinosaurus', 'aardvark', 'albatross', 'alligator']
vocab is 423 words
vocab now looks like: ['Persian', 'Spinosaurus', 'aardvark', 'albatross', 'alligator']
pmatrix has been created: (423, 423)
[[1.         0.1        0.         ... 0.         0.         0.125     ]
 [0.1        1.         0.1        ... 0.1        0.2        0.1       ]
 [0.         0.1        1.         ... 0.         0.14285714 0.        ]
 ...
 [0.         0.1        0.         ... 1.         0.         0.        ]
 [0.         0.2        0.14285714 ... 0.         1.         0.625     ]
 [0.125      0.1        0.         ... 0.         0.625      1.        ]]
phon matrix has been saved to drive!


## get frequencies

In [102]:
frequencies = pd.read_csv(parentfolder+ f'frequencies.csv', header=None)
frequencies.columns = ["word", "freq"]
# simlabels = pd.read_csv(parentfolder+'word2vec_sim_labels.csv', header=None).values.reshape(-1,).tolist() 
freq_small = pd.DataFrame([frequencies.loc[list(frequencies.word).index(lab)].freq for lab in simlabels])
freq_small['word'] = simlabels
## keep only those that are part of simlabels
freq_small.to_csv(parentfolder + 'word2vec_frequencies.csv', index=False, header=False)  

## define foraging models

For these models, the behavioral data is read in as .txt files separated by a space

In [109]:
import os

def modelFits(path, delimiter):

    ### LOAD REQUIRED PACKAGES ###
    import numpy as np
    import pandas as pd
    import re

    ### LOAD BEHAVIORAL DATA ###
    df = pd.read_csv(path, header=None, names=['SID', 'entry'], delimiter=delimiter)

    #correct behavioral fits
    df = forage.prepareData(df)

    ### LOAD SEMANTIC SIMILARITY MATRIX ###

    # (aka 'local cues', here we use cosines from word2vec)

    # Similarity labels
    simlab = []
    ofile = open(parentfolder + 'word2vec_sim_labels.csv','r')#TODO:
    for line in ofile:
        labs = line.split()
        for lab in labs:
            simlab.append(lab)
    ofile.close()

    # Similarity values
    simval = np.zeros((len(simlab), len(simlab)))
    ofile = open(parentfolder + 'word2vec_sim_matrix.csv', 'r')#TODO:
    j=0
    for line in ofile:
        line = re.sub(',\n', '', line)
        sims = line.split(',')
        i=0
        for sim in sims:
            simval[i,j] = sim
            i+=1
        j+=1
    ofile.close()

    # Make sure similarity values are non-zero
    for i in range(0,len(simval)):
        for j in range(0,len(simval)):
            if simval[i,j] <= 0:
                simval[i,j] = 0.0001

    ## PHONEMIC SIMILARTY VALUES ##
    phonval = np.zeros((len(simlab), len(simlab)))
    ofile = open(parentfolder + 'simlabels_phon_matrix.csv', 'r')#TODO:
    j=0
    for line in ofile:
        line = re.sub(',\n', '', line)
        sims = line.split(',')
        i=0
        for sim in sims:
            phonval[i,j] = sim
            i+=1
        j+=1
    ofile.close()

    # Make sure phonemic values are non-zero
    for i in range(0,len(phonval)):
        for j in range(0,len(phonval)):
            if phonval[i,j] <= 0:
                phonval[i,j] = 0.0001

    ### LOAD FREQUENCY LIST ###
    # (aka 'global cue', using NOW corpus from http://corpus.byu.edu/now/, 4.2 billion words and growing daily)

    freqlab = []
    freqval = []
    ofile = open(parentfolder + 'word2vec_frequencies.csv', 'r') #TODO:
    for line in ofile:
        line = re.sub('\n', '', line)
        freqs=line.split(',')
        freqlab.append(freqs[1])
        ## append log of frequency if using psyrev
        freqval.append(np.log(float(freqs[0])))
        #freqval.append(float(freqs[1]))
    ofile.close()
    freqval=np.array(freqval)

    sidlist = list(set(df['SID']))
    full_entdf = pd.DataFrame()
    full_fitlist = []
    ct = 0

    ## COMPUTE CONSECUTIVE SIMILARITY AND FREQUENCY AT SUBJECT LEVEL ##

    for sid in sidlist:
        ct+=1
        print( "SUBJECT " + str(ct) + '/' + str(len(sidlist)) + " " + str(sid))

        # My general initializations
        myfitlist = []
        myentries = np.array(df[df['SID']==sid]['entry'])
        #print("myentries:", myentries)
        myenttimes = np.array(df[df['SID']==sid].index)
        ##print("myenttimes:", myenttimes)
        myused = []
        mytime = []

        # For both frequency and similarity metrics:
            # LIST: Metrics corresponding with my observed entries
            # CURRENT: Full metric values, with observed entries becoming 0
            # HISTORY: State of full metric values (ie, "current" during each entry)

        # My frequency initializations
        # freq current contains frequencies of ALL the words in corpus
        freq_current = np.array(freqval)
        #print("freq_current.shape:",freq_current.shape)
        freq_list = []
        freq_history = []

        # My similarity initializations
        sim_current = simval.copy()
        # sim_current contains the full NxN similarity matrix
        #print("sim_current shape:",sim_current.shape)
        sim_list = []
        sim_history = []

        phon_current = phonval.copy()
        phon_list = []
        phon_history = []

        for i in range(0,len(myentries)):
            word = myentries[i]
            #if word not in myused: # use this to calculate number of correct responses w/out repeats
            if True:   # use this line instead of former to include repeated words along w/line 110,119 comment out

                # Frequency: Get frequency and update relevant lists
                freq_list.append( float(freq_current[freqlab.index(word)]) )
                freq_history.append( np.array(freq_current) )
                #freq_current[freqlab.index(word)] = 0.00000001

                # Get similarity between this word and preceding word
                if i > 0:         
                    sim_list.append( float(sim_current[simlab.index(myentries[i-1]), simlab.index(word)]) )
                    sim_history.append( np.array(sim_current[simlab.index(myentries[i-1]),:]) )

                    phon_list.append( float(phon_current[simlab.index(myentries[i-1]), simlab.index(word)]) )
                    phon_history.append( np.array(phon_current[simlab.index(myentries[i-1]),:]) )
                else:
                    sim_list.append(0)
                    sim_history.append( np.array(sim_current[simlab.index(word),:]) )
                #sim_current[:,simlab.index(word)] = 0.00000001
                    phon_list.append(0)
                    phon_history.append( np.array(phon_current[simlab.index(word),:]) )

                # Update lists
                myused.append(word)
                mytime.append(myenttimes[i])

        # Calculate category switches, based on similarity-drop
        myswitch = np.zeros(len(myused)).astype(int)
        for i in range(1,len(myused)-1):
            if (sim_list[i+1] > sim_list[i]) and (sim_list[i-1] > sim_list[i]):
                myswitch[i] = 1

        # Save my entries with corresponding metrics
        mydf = pd.DataFrame({'sid':[sid]*len(myused) , 'ent':myused, 'freq':freq_list, 'sim':sim_list, 'phon': phon_list,
                             'switch':myswitch, 'time':mytime},
                            columns=['sid','time','ent','freq','sim', 'phon', 'switch'])
        full_entdf = full_entdf.append(mydf)
        # Get parameter fits for the different models
        myfitlist.append(sid)
        myfitlist.append(len(myused))
        ## obtaining the optimal/random fits for the static and dynamic model by calling the getFits function
        myfitlist.extend( forage.getfits(freq_list, freq_history, sim_list, sim_history, phon_list, phon_history) )
        full_fitlist.append(myfitlist)

    print("Fits Complete.")

    # create results directory if it doesn't exist yet
    if not os.path.exists(parentfolder):
        os.makedirs(parentfolder)

    # # Output data entries with corresponding metrics for visualization in R
    print(full_entdf)
    full_entdf = full_entdf.reset_index(drop=True)
    full_entdf.to_csv(parentfolder  + 'nancy-fullmetrics.csv', index=False, header=True)

    # # Output parameter & model fits
    full_fitlist = pd.DataFrame(full_fitlist)
    full_fitlist.columns = ['subject', 'number_of_items', 
                            'beta_static_frequency', 'beta_static_semantic', 'errors_static_optimal', 'errors_static_random',
                            'beta_dynamic_frequency', 'beta_dynamic_semantic', 'errors_dynamic_optimal', 'errors_dynamic_random',
                            'beta_dynamicjack_frequency', 'beta_dynamicjack_semantic', 'errors_dynamicjack_optimal', 'errors_dynamicjack_random',

                            'beta_plocalstatic_frequency', 'beta_plocalstatic_semantic', 'beta_plocalstatic_phonemic','errors_plocalstatic_optimal', 'errors_plocalstatic_random',

                            'beta_plocaldynamicorig_frequency', 'beta_plocaldynamicorig_semantic', 'beta_plocaldynamicorig_phonemic','errors_plocaldynamicorig_optimal', 'errors_plocaldynamicorig_random',
                            'beta_pglobaldynamicorig_frequency', 'beta_pglobaldynamicorig_semantic', 'beta_pglobaldynamicorig_phonemic','errors_pglobaldynamicorig_optimal', 'errors_pglobaldynamicorig_random',

                            'beta_plocaldynamicjack_frequency', 'beta_plocaldynamicjack_semantic', 'beta_plocaldynamicjack_phonemic','errors_plocaldynamicjack_optimal', 'errors_plocaldynamicjack_random',
                            'beta_pglobaldynamicjack_frequency', 'beta_pglobaldynamicjack_semantic', 'beta_pglobaldynamicjack_phonemic','errors_pglobaldynamicjack_optimal', 'errors_pglobaldynamicjack_random',

                            'beta_pswitchonlydynamicjack_frequency', 'beta_pswitchonlydynamicjack_semantic', 'beta_pswitchonlydynamicjack_phonemic','errors_pswitchonlydynamicjack_optimal', 'errors_pswitchonlydynamicjack_random'
                            ]

    #print("full_fitlist:",full_fitlist)
    full_fitlist.to_csv(parentfolder  + 'nancy-fullfits.csv', index=False, header=True)

    print(full_fitlist.head())
    print("Results saved to '" + parentfolder + "'.")

class forage:

    def prepareData(data):
        import pandas as pd
        import re
        # load similarity labels
        simlab = []
        ofile = open(parentfolder + 'word2vec_sim_labels.csv','r')
        for line in ofile:
            labs = line.split()
            for lab in labs:
                simlab.append(lab)
        ofile.close()

        ### LOAD CORRECTIONS ###
        # This is a look-up list that maps incorrect words onto accepted words that are in the database
        corrections = pd.read_csv(parentfolder + 'corrections.txt', header=None, delimiter='\t')
        corrections = corrections.set_index(corrections[0].values)
        corrections.columns = ['_from','_to']

        elist = data['entry'].values
        newlist = []
        notfound = []

        # Use look-up table to check and correct observed entries
        for ent in elist:
            ent = re.sub(r'\W+', '', ent) # Alphanumericize it
            if ent in simlab:
                # If this entry is appropriate, keep it
                newlist.append(ent)
            elif ent[0:len(ent)-1] in simlab:
                # If this entry is plural, correct to the singular verion
                print(f"found the entry {ent[0:len(ent)-1]} in simlab")
                newlist.append(ent[0:len(ent)-1])
            # elif ent in corrections._from:
            #     # If this entry is correctable, correct it
            #     newlist.append(corrections.loc[ent]._to)
            else:
                # If this entry is not found in either list, mark for removal and warn user.
                newlist.append('NA')
                notfound.append(ent)

        # Remove the rows with inappropriate entries
        data.entry = newlist
        data = data[data.entry!='NA']

        # Warn the user of removed entries
        if len(notfound) > 0:
            print('The following items were not found in the database, and were removed: [' +
                  str(len(notfound)) + ' entries removed] \n')
            print(sorted(set(notfound)))
        else:
            print('All items OK.')
        return data[data.entry!='NA']
        # TODO: return statement might not be necessary...

    def model_static(beta, freql, freqh, siml, simh):
        ## beta contains the optimization parameters for frequency (beta[0]) and semantic similarity (beta[1])
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0: # if first item then its probability is based on just frequency
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            else: # if not first item then its probability is based on its similarity to prev item AND frequency
            # P of item based on frequency and similarity
                numrat = pow(freql[k],beta[0]) * pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0]) * pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat) # negative Log likelihood of this item: this will be minimized eventually
        return ct


    def model_dynamic_original(beta, freql, freqh, siml, simh):
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif k > 0 and k < (len(freql)-1) and siml[k+1] > siml[k] and siml[k-1] > siml[k]: ## "dip" based on sim-drop
            # If similarity dips, P of item is based again on frequency
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(siml[k],beta[1])
                denrat = sum(pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct
        
    def model_dynamic_jack(beta, freql, freqh, siml, simh):
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif k > 0 and k < (len(freql)-1) and siml[k+1] > siml[k] and siml[k-1] > siml[k]: ## "dip" based on sim-drop
            # If similarity dips, P of item is based again on frequency
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(freql[k],beta[0])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct
      
    def model_static_plocal(beta, freql, freqh, siml, simh, phonl, phonh):
        ## beta contains the optimization parameters for frequency (beta[0]) and semantic similarity (beta[1])
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0: # if first item then its probability is based on just frequency
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            else: # if not first item then its probability is based on its similarity to prev item AND frequency AND phonemic similarity
            # P of item based on frequency and similarity and phonology
                numrat = pow(freql[k],beta[0]) * pow(phonl[k],beta[2]) * pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0]) * pow(phonh[k],beta[2])* pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat) # negative Log likelihood of this item: this will be minimized eventually
        return ct

    def model_dynamic_plocal_jack(beta, freql, freqh, siml, simh, phonl, phonh):
      ## here we use phonology as a "local" cue with semantics
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif k > 0 and k < (len(freql)-1) and siml[k+1] > siml[k] and siml[k-1] > siml[k]: ## "dip" based on sim-drop
            # If similarity dips, P of item is based on frequency 
                numrat = pow(freql[k],beta[0]) 
                denrat = sum(pow(freqh[k],beta[0]))
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(freql[k],beta[0])*pow(phonl[k],beta[2])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(phonh[k],beta[2])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct

    def model_dynamic_pglobal_jack(beta, freql, freqh, siml, simh, phonl, phonh):
      ## here we use phonology as a "local" cue with semantics
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif k > 0 and k < (len(freql)-1) and siml[k+1] > siml[k] and siml[k-1] > siml[k]: ## "dip" based on sim-drop
            # If similarity dips, P of item is based on a combination of frequency and phonemic similarity
                numrat = pow(freql[k],beta[0]) * pow(phonl[k],beta[2]) 
                denrat = sum(pow(freqh[k],beta[0]) * pow(phonh[k],beta[2]) )
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(freql[k],beta[0])*pow(phonl[k],beta[2])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(phonh[k],beta[2])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct

    def model_dynamic_pswitchonly_jack(beta, freql, freqh, siml, simh, phonl, phonh):
      ## here we use phonology as a "local" cue with semantics
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif k > 0 and k < (len(freql)-1) and siml[k+1] > siml[k] and siml[k-1] > siml[k]: ## "dip" based on sim-drop
            # If similarity dips, P of item is based on a combination of frequency and phonemic similarity
                numrat = pow(freql[k],beta[0]) * pow(phonl[k],beta[2]) 
                denrat = sum(pow(freqh[k],beta[0]) * pow(phonh[k],beta[2]) )
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(freql[k],beta[0])*pow(siml[k],beta[1])
                denrat = sum(pow(freqh[k],beta[0])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct
    
    def model_dynamic_plocal_original(beta, freql, freqh, siml, simh, phonl, phonh):
      ## here we use phonology as a "local" cue with semantics
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif k > 0 and k < (len(freql)-1) and siml[k+1] > siml[k] and siml[k-1] > siml[k]: ## "dip" based on sim-drop
            # If similarity dips, P of item is based on frequency 
                numrat = pow(freql[k],beta[0]) 
                denrat = sum(pow(freqh[k],beta[0]))
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(phonl[k],beta[2])*pow(siml[k],beta[1])
                denrat = sum(pow(phonh[k],beta[2])*pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct

    def model_dynamic_pglobal_original(beta, freql, freqh, siml, simh, phonl, phonh):
      ## here we use phonology as a "local" cue with semantics
        import numpy as np
        ct = 0
        for k in range(0, len(freql)):
            if k == 0 :
            # P of item based on frequency alone (freq of this item / freq of all items)
                numrat = pow(freql[k],beta[0])
                denrat = sum(pow(freqh[k],beta[0]))
            elif k > 0 and k < (len(freql)-1) and siml[k+1] > siml[k] and siml[k-1] > siml[k]: ## "dip" based on sim-drop
            # If similarity dips, P of item is based on a combination of frequency and phonemic similarity
                numrat = pow(freql[k],beta[0]) * pow(phonl[k],beta[2]) 
                denrat = sum(pow(freqh[k],beta[0]) * pow(phonh[k],beta[2]) )
            else:
            # P of item based on combined frequency and similarity
                numrat = pow(siml[k],beta[1])
                denrat = sum(pow(simh[k],beta[1]))
            ct += -np.log(numrat/denrat)
        return ct

    def getfits( freq_l, freq_h, sim_l, sim_h, phon_l, phon_h ):
        import numpy as np
        from scipy.optimize import fmin
    #fmin: Uses a Nelder-Mead simplex algorithm to find the minimum of function of variables.
        r1 = np.random.rand()
        r2 = np.random.rand()
        r3 = np.random.rand()

    # STATIC MODEL (no dynamic switching, just focusing on two cues with some weights)

        # 1.) Optimize model parameters
        v = fmin(forage.model_static, [r1, r2], args=(freq_l, freq_h, sim_l, sim_h), ftol = 0.001, disp=False)
        beta_static_freq = float(v[0]) # Optimized weight for frequency cue
        beta_static_semantic = float(v[1]) # Optimized weight for similarity cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_static = forage.model_static([beta_static_freq, beta_static_semantic], freq_l, freq_h, sim_l, sim_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_static = forage.model_static([0, 0], freq_l, freq_h, sim_l, sim_h)

    # ORIGINAL DYNAMIC MODEL (switches dynamically between cues)

        # 1.) Optimize model parameters
        v = fmin(forage.model_dynamic_original, [r1,r2], args=(freq_l, freq_h, sim_l, sim_h), ftol = 0.001, disp=False)
        beta_dynamic_freq = float(v[0]) # Optimized weight for frequency cue
        beta_dynamic_semantic = float(v[1]) # Optimized weight for similarity cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_dynamic = forage.model_dynamic_original([beta_dynamic_freq, beta_dynamic_semantic], freq_l, freq_h, sim_l, sim_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_dynamic = forage.model_dynamic_original([0,0], freq_l, freq_h, sim_l, sim_h)

    # JACK DYNAMIC MODEL (switches dynamically between cues)

        # 1.) Optimize model parameters
        v = fmin(forage.model_dynamic_jack, [r1,r2], args=(freq_l, freq_h, sim_l, sim_h), ftol = 0.001, disp=False)
        beta_dynamicjack_freq = float(v[0]) # Optimized weight for frequency cue
        beta_dynamicjack_semantic = float(v[1]) # Optimized weight for similarity cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_dynamicjack = forage.model_dynamic_jack([beta_dynamic_freq, beta_dynamic_semantic], freq_l, freq_h, sim_l, sim_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_dynamicjack = forage.model_dynamic_jack([0,0], freq_l, freq_h, sim_l, sim_h)
  

    # LOCAL PHONEMIC CUE, STATIC MODEL (no dynamic switching, just focusing on two cues with some weights)

        # 1.) Optimize model parameters
        v = fmin(forage.model_static_plocal, [r1, r2, r3], args=(freq_l, freq_h, sim_l, sim_h, phon_l, phon_h), ftol = 0.001, disp=False)
        beta_static_plocal_freq = float(v[0]) # Optimized weight for frequency cue
        beta_static_plocal_semantic = float(v[1]) # Optimized weight for similarity cue
        beta_static_plocal_phonemic = float(v[2]) # Optimized weight for phonemic cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_plocalstatic = forage.model_static_plocal([beta_static_plocal_freq, beta_static_plocal_semantic, beta_static_plocal_phonemic], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_plocalstatic = forage.model_static_plocal([0, 0, 0], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)
    
    # ORIGINAL DYNAMIC PHON LOCAL MODEL (switches dynamically between cues, phonology,semantic, freq is a LOCAL cue)

        # 1.) Optimize model parameters
        v = fmin(forage.model_dynamic_plocal_original, [r1,r2, r3], args=(freq_l, freq_h, sim_l, sim_h, phon_l, phon_h), ftol = 0.001, disp=False)
        beta_dynamic_plocalorig_freq = float(v[0]) # Optimized weight for frequency cue
        beta_dynamic_plocalorig_semantic = float(v[1]) # Optimized weight for similarity cue
        beta_dynamic_plocalorig_phonemic = float(v[2]) # Optimized weight for phonemic cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_plocaldynamicorig = forage.model_dynamic_plocal_original([beta_dynamic_plocalorig_freq, beta_dynamic_plocalorig_semantic, beta_dynamic_plocalorig_phonemic], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_plocaldynamicorig = forage.model_dynamic_plocal_original([0, 0, 0], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

     # ORIGINAL DYNAMIC PHON GLOBAL MODEL (switches dynamically between cues, phonology is a GLOBAL cue with frequency)

        # 1.) Optimize model parameters
        v = fmin(forage.model_dynamic_pglobal_original, [r1,r2, r3], args=(freq_l, freq_h, sim_l, sim_h, phon_l, phon_h), ftol = 0.001, disp=False)
        beta_dynamic_pglobalorig_freq = float(v[0]) # Optimized weight for frequency cue
        beta_dynamic_pglobalorig_semantic = float(v[1]) # Optimized weight for similarity cue
        beta_dynamic_pglobalorig_phonemic = float(v[2]) # Optimized weight for phonemic cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_pglobaldynamicorig = forage.model_dynamic_pglobal_original([beta_dynamic_pglobalorig_freq, beta_dynamic_pglobalorig_semantic, beta_dynamic_pglobalorig_phonemic], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_pglobaldynamicorig = forage.model_dynamic_pglobal_original([0, 0, 0], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

      # JACK DYNAMIC PHON LOCAL MODEL (switches dynamically between cues, phonology,semantic, freq is a LOCAL cue)

        # 1.) Optimize model parameters
        v = fmin(forage.model_dynamic_plocal_jack, [r1,r2, r3], args=(freq_l, freq_h, sim_l, sim_h, phon_l, phon_h), ftol = 0.001, disp=False)
        beta_dynamic_plocaljack_freq = float(v[0]) # Optimized weight for frequency cue
        beta_dynamic_plocaljack_semantic = float(v[1]) # Optimized weight for similarity cue
        beta_dynamic_plocaljack_phonemic = float(v[2]) # Optimized weight for phonemic cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_plocaldynamicjack = forage.model_dynamic_plocal_jack([beta_dynamic_plocaljack_freq, beta_dynamic_plocaljack_semantic, beta_dynamic_plocaljack_phonemic], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_plocaldynamicjack = forage.model_dynamic_plocal_jack([0, 0, 0], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

     # JACK DYNAMIC PHON GLOBAL MODEL (switches dynamically between cues, phonology is a GLOBAL cue with frequency)

        # 1.) Optimize model parameters
        v = fmin(forage.model_dynamic_pglobal_jack, [r1,r2, r3], args=(freq_l, freq_h, sim_l, sim_h, phon_l, phon_h), ftol = 0.001, disp=False)
        beta_dynamic_pglobaljack_freq = float(v[0]) # Optimized weight for frequency cue
        beta_dynamic_pglobaljack_semantic = float(v[1]) # Optimized weight for similarity cue
        beta_dynamic_pglobaljack_phonemic = float(v[2]) # Optimized weight for phonemic cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_pglobaldynamicjack = forage.model_dynamic_pglobal_jack([beta_dynamic_pglobaljack_freq, beta_dynamic_pglobaljack_semantic, beta_dynamic_pglobaljack_phonemic], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_pglobaldynamicjack = forage.model_dynamic_pglobal_jack([0, 0, 0], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)
      
    # JACK DYNAMIC PHON SWITCH ONLY MODEL (switches dynamically between cues, phonology is a GLOBAL cue with frequency)

        # 1.) Optimize model parameters
        v = fmin(forage.model_dynamic_pswitchonly_jack, [r1,r2, r3], args=(freq_l, freq_h, sim_l, sim_h, phon_l, phon_h), ftol = 0.001, disp=False)
        beta_dynamic_pswitchonlyjack_freq = float(v[0]) # Optimized weight for frequency cue
        beta_dynamic_pswitchonlyjack_semantic = float(v[1]) # Optimized weight for similarity cue
        beta_dynamic_pswitchonlyjack_phonemic = float(v[2]) # Optimized weight for phonemic cue

        # 2.) Determine model fit (errors) at optimal parameters: will return total -LL
        optimal_fit_pswitchonlydynamicjack = forage.model_dynamic_pswitchonly_jack([beta_dynamic_pswitchonlyjack_freq, beta_dynamic_pswitchonlyjack_semantic, beta_dynamic_pswitchonlyjack_phonemic], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)

        # 3.) For comparison, determine model fit (errors) without parameter fits
        random_fit_pswitchonlydynamicjack = forage.model_dynamic_pswitchonly_jack([0, 0, 0], freq_l, freq_h, sim_l, sim_h, phon_l, phon_h)
      
      

        results = [ beta_static_freq, beta_static_semantic, float(optimal_fit_static), float(random_fit_static),
                   beta_dynamic_freq, beta_dynamic_semantic, float(optimal_fit_dynamic), float(random_fit_dynamic),
                   beta_dynamicjack_freq, beta_dynamicjack_semantic, float(optimal_fit_dynamicjack), float(random_fit_dynamicjack),

                   beta_static_plocal_freq, beta_static_plocal_semantic, beta_static_plocal_phonemic, float(optimal_fit_plocalstatic), float(random_fit_plocalstatic),
              
                   beta_dynamic_plocalorig_freq, beta_dynamic_plocalorig_semantic, beta_dynamic_plocalorig_phonemic, float(optimal_fit_plocaldynamicorig), float(random_fit_plocaldynamicorig),
                   beta_dynamic_pglobalorig_freq, beta_dynamic_pglobalorig_semantic, beta_dynamic_pglobalorig_phonemic, float(optimal_fit_pglobaldynamicorig), float(random_fit_pglobaldynamicorig),

                   beta_dynamic_plocaljack_freq, beta_dynamic_plocaljack_semantic, beta_dynamic_plocaljack_phonemic, float(optimal_fit_plocaldynamicjack), float(random_fit_plocaldynamicjack),
                   beta_dynamic_pglobaljack_freq, beta_dynamic_pglobaljack_semantic, beta_dynamic_pglobaljack_phonemic, float(optimal_fit_pglobaldynamicjack), float(random_fit_pglobaldynamicjack),

                   beta_dynamic_pswitchonlyjack_freq, beta_dynamic_pswitchonlyjack_semantic, beta_dynamic_pswitchonlyjack_phonemic, float(optimal_fit_pswitchonlydynamicjack), float(random_fit_pswitchonlydynamicjack)

                   ]

        return results

## run foraging models

In [110]:
datapath = parentfolder+'data-psyrev.txt'
modelFits(datapath, delimiter = " ")

found the entry gull in simlab
found the entry lizard in simlab
The following items were not found in the database, and were removed: [248 entries removed] 

['akita', 'antlion', 'axolotl', 'bacterium', 'bengaltiger', 'billygoat', 'blackbear', 'blackmamba', 'blacktaileddeer', 'blackwidow', 'blowfish', 'bluefootedbooby', 'bluejay', 'bluewhale', 'boaconstrictor', 'brineshrimp', 'bronco', 'brownbear', 'cattle', 'dalmation', 'duckbill', 'gardensnake', 'goldenmarmot', 'greatdane', 'greatwhiteshark', 'guineapig', 'hammerheadshark', 'humpbackwhale', 'jackrabbit', 'junebug', 'killerwhale', 'kodiakbear', 'komododragon', 'labradorretriever', 'liger', 'makoshark', 'malamute', 'mantaray', 'mountainbear', 'mountaingoat', 'mountainlion', 'muledeer', 'okapi', 'pembrokewelshcorgi', 'pike', 'pillbug', 'polarbear', 'potbelliedpig', 'prairiedog', 'prayingmantis', 'puggle', 'redrumpedagouti', 'rhinoceros', 'sanddollar', 'sealion', 'seamonkey', 'shihtzu', 'siamesecat', 'siberianhusky', 'snowfox', 'snowleop